In [2]:
import kagglehub
kagglehub.login()
kagglehub.dataset_download('hlgsagar1234567/vr-go')
kagglehub.dataset_download("adityaav80/validatiioon")

'/kaggle/input/validatiioon'

In [14]:
import pandas as pd
df = pd.read_csv("/kaggle/input/validatiioon/val.csv")

print(len(df))
df.head(5)

106328


,image_id,image_path,question,answer
0,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,What is the main color of the phone case?,Black
1,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,What is the object depicted on the phone case?,UFO
2,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,Where is the camera lens positioned on the pho...,Top
3,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,How many stars are visible around the UFO on t...,Seven
4,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,What is the phone case covering?,Phone


In [5]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

# Load processor & model (with trust_remote_code to pick up the custom BLIP2 code)
processor = Blip2Processor.from_pretrained(
    "Salesforce/blip2-opt-2.7b", 
    trust_remote_code=True,
    use_fast=True
)
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

2025-05-12 15:31:28.469803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747063888.654586      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747063888.708005      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
new_model = model.to(device)

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import re
import os
import transformers
import matplotlib.pyplot as plt

def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', str(text).lower().strip())


st,en = 0,1
start_row = int(106328/10)*st+1
end_row =  int(106328/10)*en
output_csv = f"/kaggle/working/evaluation_{st}_{en}.csv"

baai = SentenceTransformer("BAAI/bge-base-en-v1.5")

results = []

for i in range(start_row, min(end_row, len(df))):
    try:
        if i % 10 == 0:
            image_path = df.loc[i, "image_path"]
            image = Image.open(image_path).convert("RGB")

            plt.show(image)

        question = df.loc[i, "question"]
        gt = df.loc[i, "answer"]

        prompt1 = f"Question:{question}. Answer in **one word** only. Answer:"
        prompt2 = f"Question: {question}. Give a complete and accurate. Answer:"

        with torch.no_grad():
            inputs_short = processor(image, text=prompt1, return_tensors="pt").to(device, torch.float16)
            pred_ids_short = model.generate(**inputs_short, max_new_tokens=5)
            pred_short = processor.batch_decode(pred_ids_short, skip_special_tokens=True)[0].split("Answer:")[-1].strip()

            inputs_long = processor(image, text=prompt2, return_tensors="pt").to(device, torch.float16)
            pred_ids_long = model.generate(**inputs_long, max_new_tokens=50)
            pred_long = processor.batch_decode(pred_ids_long, skip_special_tokens=True)[0].split("Answer:")[-1].strip()

            torch.cuda.empty_cache()

        pred_short_clean = clean_text(pred_short)
        pred_long_clean = clean_text(pred_long)
        gt_clean = clean_text(str(gt))

        # Compute similarity only if both GT and prediction exist
        if pred_short_clean and gt_clean:
            emb_short = baai.encode(pred_short_clean, convert_to_tensor=True, show_progress_bar=False)
            emb_gt = baai.encode(gt_clean, convert_to_tensor=True)
            sim_short = min(1.0, round(util.cos_sim(emb_short, emb_gt).item(), 3))
        else:
            sim_short = ""
        
        if pred_long_clean and gt_clean:
            emb_long = baai.encode(pred_long_clean, convert_to_tensor=True, show_progress_bar=False)
            emb_gt = baai.encode(gt_clean, convert_to_tensor=True)
            sim_long = min(1.0, round(util.cos_sim(emb_long, emb_gt).item(), 3))
        else:
            sim_long = ""
        
        # Exact match only if both exist
        exact_match = 1 if pred_short_clean and gt_clean and (pred_short_clean == gt_clean) else 0                    

        results.append({
            "index": i,
            "image_path": image_path,
            "question": question,
            "answer": gt,
            "blip2_prediction_short": pred_short_clean,
            "blip2_prediction_long": pred_long_clean,
            "baai_similarity_short": sim_short,
            "baai_similarity_long": sim_long,
            "blip2_exact_match_short": exact_match
        })

        if (i - start_row + 1) % 1000 == 0:
            pd.DataFrame(results).to_csv(output_csv, mode='a', index=False, header=not os.path.exists(output_csv))
            print(f"[SAVED] Up to row {i}")
            results = []

    except Exception as e:
        print(f"[ERROR] at row {i}: {e}")
        continue

# Final save
if results:
    pd.DataFrame(results).to_csv(output_csv, mode='a', index=False, header=not os.path.exists(output_csv))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

# Run in case of CUDA ERROR

In [ ]:
import gc
import torch

# Delete your variables (models, data, etc.)
try:
    del model
except:
    print("No model")
# or any other large variable like dataloader, optimizer, etc.
try:
    del processor
except:
    print("No processor")
# Run garbage collector
gc.collect()

# Empty the PyTorch GPU cache
torch.cuda.empty_cache()

# Forcing a GPU memory clean-up (optional, may help)
torch.cuda.ipc_collect()


!rm -rf ~/.cache/huggingface
!rm -rf ~/.cache/torch
!rm -rf ~/.cache/evaluate
!rm -rf ~/.cache/transformers
!rm -rf /kaggle/working/*
!rm -rf /kaggle/temp/*